In this notebook, I test the GCPStrokeModel against the results obtained during the developement of this model. First, I reproduce the results by using the appropriate mathematical expression. I then reproduce the results using the method calc_linear_predictor_for_patient_characteristics without using any person objects, followed by the same thing but using a person object. I finally test the reproducibility of the results using a person object and the get_risk_for_person non vectorized approach and then the vectorized approach.

In [1]:
import unittest
import numpy as np
import operator
import pandas as pd
pd.set_option('display.max_columns', None)
#pd.set_option('max_columns', None)
pd.set_option('display.max_rows', None)
from microsim.person import Person
from microsim.population import Population
from microsim.education import Education
from microsim.gender import NHANESGender
from microsim.smoking_status import SmokingStatus
from microsim.alcohol_category import AlcoholCategory
from microsim.race_ethnicity import NHANESRaceEthnicity
from microsim.gcp_model import GCPModel
from microsim.gcp_stroke_model import GCPStrokeModel
from microsim.outcome import Outcome, OutcomeType
from microsim.test.do_not_change_risk_factors_model_repository import (
    DoNotChangeRiskFactorsModelRepository,
)
from microsim.outcome_model_repository import OutcomeModelRepository

### REPRODUCE RESULTS USING JUST MATH

In [2]:
#read the file with the results
peopleAim1 = pd.read_csv(
    "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM/PAPERS/STROKE COG_Aim-1_Exemplars.csv")

#peopleAim1.head(5)

In [3]:
#yhat is what the model predicts, so yhat+residual = fgcp
(peopleAim1["yhat"]+peopleAim1["resid"] - (peopleAim1["fgcp"])).head()

0    3.100000e-07
1    2.210000e-06
2   -8.000000e-07
3    2.210000e-06
4   -4.790000e-06
dtype: float64

In [4]:
#include the time dependent variables of the model
peopleAim1["sbpmed10*t_gcp_stk"] = peopleAim1["t_gcp_stk"]*peopleAim1["sbpmed10"]
peopleAim1["htntx*t_gcp_stk"] = peopleAim1["t_gcp_stk"]*peopleAim1["htntx"]
peopleAim1["glucosefmed10*t_gcp_stk"] = peopleAim1["t_gcp_stk"]*peopleAim1["glucosefmed10"]
peopleAim1["diabetestx*t_gcp_stk"] = peopleAim1["t_gcp_stk"]*peopleAim1["diabetestx"]
peopleAim1["cholldlmed10*t_gcp_stk"] = peopleAim1["t_gcp_stk"]*peopleAim1["cholldlmed10"]
peopleAim1["choltx*t_gcp_stk"] = peopleAim1["t_gcp_stk"]*peopleAim1["choltx"]
peopleAim1["female0*t_gcp_stk"] = peopleAim1["t_gcp_stk"]*peopleAim1["female0"]
peopleAim1["agemed10*t_gcp_stk"] = peopleAim1["t_gcp_stk"]*peopleAim1["agemed10"]
peopleAim1["reffect1*t_gcp_stk"] = peopleAim1["t_gcp_stk"]*peopleAim1["reffect1"]

In [5]:
#add a row with the model coefficients, in the order the columns are
peopleAim1.columns

Index(['newid', 'fgcp', 'sbpmed10', 'cholldlmed10', 'glucosefmed10',
       't_gcp_stk', 'hxafib', 'alcperwk', 'choltx', 'currsmoker', 'diabetestx',
       'gfr', 'htntx', 'hxmi', 'physact', 'educ2', 'educ3', 'educ4', 'income8',
       'income2', 'income3', 'income4', 'income5', 'bs_glucosefmed10',
       'bs_cholldlmed10', 'bs_sbpstkcogmed10', 'bs_waistcmmed10', 'bs_fgcpmed',
       'bs_bmimed', 'female0', 'black', 'agemed10', 'aric', 'chs', 'fos',
       'yhat', 'resid', 'reffect1', 'reffect2', 'sbpmed10*t_gcp_stk',
       'htntx*t_gcp_stk', 'glucosefmed10*t_gcp_stk', 'diabetestx*t_gcp_stk',
       'cholldlmed10*t_gcp_stk', 'choltx*t_gcp_stk', 'female0*t_gcp_stk',
       'agemed10*t_gcp_stk', 'reffect1*t_gcp_stk'],
      dtype='object')

In [6]:
#I got these from the output file we got from colleagues
peopleAim1.loc["coeffs"] = ["coeffs",
                            1.,      #fgcp
                            0.2300907,  #sbpmed10
                            -0.0912223, #cholldlmed10
                            0.1561969,  #glucosefmed10
                            -0.5249359, #t_gcp_stk
                            -1.532905, #hxafi
                            0.0550249, #alcperwk
                            -0.7570241, #choltx
                            0.4706579,  #currsmoker
                            -1.46007, #diabetest
                            0.0175139,  #gfr
                            -1.3711, #htntx
                            0.4470,  #hxmi
                            1.0047,  #physact
                            0.9930,  #educ2
                            -0.00267,#educ3
                            0.5712,  #educ4
                            0.05448, #income8
                            0.4819,  #income2
                            1.0150,  #income3
                            1.7918,  #income4
                            1.9204,  #income5
                            -0.02933,#bs_glucosefmed10
                            -0.1866, #bs_cholldlmed10
                            -0.2535435, #bs_sbpstkcogmed10
                            0.2726,  #bs_waistcmmed10
                            0.663,   #bs_fgcpmed
                            -0.1372, #bs_bmimed
                            1.4858,  #female0
                            -1.5739, #black
                            -1.4919, #agemed10
                            -5.2897, #aric
                            -3.7359, #chs
                            -2.8168, #fos
                            0.,      #yhat
                            1,       #resid
                            0.,      #reffect1
                            1.,      #reffect2
                            0.0424804, #sbpmed10*t_gcp_stk
                            0.2271476, #htntx*t_gcp_stk
                            -0.042661,  #glucosefmed10*t_gcp_stk
                            -0.0378811, #diabetestx*t_gcp_stk
                            0.007825,    #cholldlmed10*t_gcp_stk
                            0.1034871,   #choltx*t_gcp_stk
                            -0.2863992,  #female0*t_gcp_stk
                            -0.1970112,  #agemed10*t_gcp_stk
                             1.]         #reffect1*t_gcp_stk

In [7]:
#calculate the gcp  I predict and find the difference with the gcp I should be getting
myFgcp = []
myFgcpDiff = []
for j in range(peopleAim1.shape[0]-1):
    productSr = (peopleAim1.iloc[peopleAim1.shape[0]-1,[i for i in range(2,48)]] * 
                                 peopleAim1.iloc[j,[i for i in range(2,48)]])
    myFgcp = myFgcp + [productSr.sum()  + 51.9602]
    myFgcpDiff = myFgcpDiff + [myFgcp[j] - peopleAim1.iloc[j]["fgcp"]]
peopleAim1["myFgcpDiff"] = myFgcpDiff + [0]
peopleAim1["myFgcp"] = myFgcp + [0]

In [8]:
peopleAim1["myFgcpDiff"].describe()

count    44.000000
mean     -0.001190
std       0.000935
min      -0.002569
25%      -0.002049
50%      -0.001276
75%      -0.000848
max       0.001058
Name: myFgcpDiff, dtype: float64

In [9]:
#this helps with troubleshooting...
#for each term that goes in the model, add a column with the value of that term

covslist = ['sbpmed10', 'cholldlmed10', 'glucosefmed10',
       't_gcp_stk', 'hxafib', 'alcperwk', 'choltx', 'currsmoker', 'diabetestx',
       'gfr', 'htntx', 'hxmi', 'physact', 'educ2', 'educ3', 'educ4', 'income8',
       'income2', 'income3', 'income4', 'income5', 'bs_glucosefmed10',
       'bs_cholldlmed10', 'bs_sbpstkcogmed10', 'bs_waistcmmed10', 'bs_fgcpmed',
       'bs_bmimed', 'female0', 'black', 'agemed10', 'aric', 'chs', 'fos',
       'reffect1', 'reffect2', 'sbpmed10*t_gcp_stk',
       'htntx*t_gcp_stk', 'glucosefmed10*t_gcp_stk', 'diabetestx*t_gcp_stk',
       'cholldlmed10*t_gcp_stk', 'choltx*t_gcp_stk', 'female0*t_gcp_stk',
       'agemed10*t_gcp_stk', 'reffect1*t_gcp_stk']

for col in covslist:
    peopleAim1[col+"Term"] = peopleAim1.loc[:,col] * peopleAim1.loc["coeffs",col]

#and a column with the sum of the terms (I am not including the residual here, so this is just yhat)
peopleAim1["sumOfTerms"] = (peopleAim1.loc[:, [col+"Term" for col in covslist]].sum(axis=1) +
                            51.9602)

#peopleAim1[["newid","t_gcp_stk","myFgcpDiff","myFgcp", "fgcp","yhat","sumOfTerms"]].head()

### REPRODUCE RESULTS USING calc_linear_predictor_for_patient_characteristics AND THE DATAFRAME

In [10]:
#need to re-create the original measurements, I used the publication to find the means and medians
peopleAim1["age"] = peopleAim1["agemed10"]*10 + 74.6
peopleAim1["bs_bmi"] = peopleAim1["bs_bmimed"] + 27.2
peopleAim1["bs_waistcm"] = peopleAim1["bs_waistcmmed10"]*10 + 97.5
peopleAim1["sbp"] = peopleAim1["sbpmed10"]*10 + 134.9
peopleAim1["glucosef"] = peopleAim1["glucosefmed10"]*10 +108.1
peopleAim1["bs_glucosef"] = peopleAim1["bs_glucosefmed10"]*10 + 112.8
peopleAim1["bs_fgcp"] = peopleAim1["bs_fgcpmed"]+52.7
peopleAim1["bs_sbpstkcog"] = peopleAim1["bs_sbpstkcogmed10"]*10 + 140.4
peopleAim1["cholldl"] = peopleAim1["cholldlmed10"]*10 + 94.1
peopleAim1["bs_cholldl"] = peopleAim1["bs_cholldlmed10"]*10+126.4

In [11]:
#define a few functions
def getEducation(row):
    if row["educ2"]==1.:
        return Education.HIGHSCHOOLGRADUATE
    elif row["educ3"]==1.:
        return Education.SOMECOLLEGE
    elif row["educ4"]==1.:
        return Education.COLLEGEGRADUATE
    else:
        return Education.SOMEHIGHSCHOOL
    
def getGender(row):
    if row["female0"]==1.:
        return NHANESGender.FEMALE
    else:
        return NHANESGender.MALE
    
def getRace(row):
    if row["black"]==1.:
        return NHANESRaceEthnicity.NON_HISPANIC_BLACK
    else:
        return NHANESRaceEthnicity.NON_HISPANIC_WHITE
    
def getIncome(row):
    if row["income8"]==1.:
        return 0.05448
    elif row["income2"]==1.:
        return 0.4819
    elif row["income3"]==1.:
        return 1.0150
    elif row["income4"]==1.:
        return 1.7918
    elif row["income5"]==1.:
        return 1.9204
    else:
        return 0.
    
def getGroup(row):
    if row["aric"]==1.:
        return -5.2897
    elif row["chs"]==1.:
        return -3.7359
    elif row["fos"]==1.:
        return -2.8168
    else:
        return 0.
    
def getSmoking(row):
    if row["currsmoker"]==1.:
        return SmokingStatus.CURRENT
    else:
        return SmokingStatus.NEVER
    
def getDiabetesTx(row):
    if row["diabetestx"]==1.:
        return (-1.4601 + (-0.03788)*row["t_gcp_stk"])
    else:
        return 0.
    
def getAlcperwk(row):
    return row["alcperwk"]*0.0550249

In [12]:
#choose a person to test
personAim1 = peopleAim1.iloc[0]
row=personAim1

In [13]:
# LP = linear predictor
def getLP(row):
    return GCPStrokeModel().calc_linear_predictor_for_patient_characteristics(
    ageAtLastStroke=row.age,
    yearsSinceStroke=row.t_gcp_stk,
    gender=getGender(row), #5-7
    raceEthnicity=getRace(row),
    education=getEducation(row), #5-7
    smokingStatus=getSmoking(row),
    #diabetes=row.diabetestx,
    physicalActivity=row.physact,
    #include this term manually at the end, microsim treats this as categorical    
    alcoholPerWeek=0., #row.alcperwk, 
    medianBmiPrestroke=row.bs_bmi,
    meanSBP=row.sbp,
    meanSBPPrestroke=row.bs_sbpstkcog,
    meanLdlPrestroke=row.bs_cholldl,
    meanLdl=row.cholldl,
    gfr=row.gfr,
    medianWaistPrestroke=row.bs_waistcm,
    meanFastingGlucose=row.glucosef,
    meanFastingGlucosePrestroke=row.bs_glucosef,
    anyAntiHypertensive=row.htntx,
    anyLipidLowering=row.choltx,
    afib=row.hxafib,
    mi=row.hxmi,
    medianGCPPrestroke=row.bs_fgcp
) 

def getMyYhat(row):
    return (getLP(row) 
            - (238.*(-5.2897)+332.*(-3.7359)+101.*(-2.8168)) / (238.+332.+101.+311.) #removegroupaverage
            + getGroup(row)   #get correct group contribution
            + getIncome(row)  #get income contribution
            + getDiabetesTx(row) #get diabetes treatment contribution
            + getAlcperwk(row) #get alcperwk
            + row.reffect2    
            + row["reffect1*t_gcp_stk"])

In [14]:
peopleAim1["myYhat"] = peopleAim1.apply( lambda row: getMyYhat(row), axis=1)
peopleAim1["yhatDiff"] = peopleAim1["myYhat"] - peopleAim1["yhat"]

#do not include the last row, because that row includes the coefficients
peopleAim1.iloc[0:-1]["yhatDiff"].describe()

count    43.000000
mean     -0.000091
std       0.000404
min      -0.001100
25%      -0.000363
50%       0.000071
75%       0.000210
max       0.000408
Name: yhatDiff, dtype: float64

Now that getLP is validated, I will use the getLP result to validate the rest.

### REPRODUCE RESULTS USING calc_linear_predictor_for_patient_characteristics AND A PERSON

In [15]:
#this is what I got before with the dataframe (original measurements) so I need to reproduce 
#this with the person object
getLP(peopleAim1.iloc[0])

51.87570726170979

In [16]:
#use this for initialization of the person
class TestGCPStrokeModel():
    def initializeAfib(person):
        return None

In [17]:
#these are the values for a single person, first row of the excel file
#I am using to create a simulation person that at some point had a stroke during the simulation
ageAtStroke = 74.6 + 0.389117043*10 
indexStroke = 2
ageList = [ageAtStroke + 3.17 + x for x in range(-5,1)]
ageList.insert(indexStroke, ageAtStroke)
sbpMeanPrestroke = 140.4 -1.9*10
sbpMean = -1.*10 +134.9
sbpList = [sbpMeanPrestroke] * (indexStroke+1) + [sbpMean]*4
dbpList = [80]*7
a1cMeanPrestroke = Person.convert_fasting_glucose_to_a1c(112.8 + 0.5*10)
a1cMean = Person.convert_fasting_glucose_to_a1c(108.1 + 2.2*10)
a1cList = [a1cMeanPrestroke] * (indexStroke+1) + [a1cMean]*4
hdlList = [50]*7
ldlMeanPrestroke = 126.4 + 3.8*10
ldlMean = 2.6*10 + 94.1
ldlList = [ldlMeanPrestroke] * (indexStroke+1) + [ldlMean]*4
trigList = [150]*7
totCholList = [hdlList[i]+ldlList[i]+trigList[i] for i in range(len(ldlList))]
bmiMedianPrestroke = 27.2 + 1.467401286
bmiList = [bmiMedianPrestroke] * (indexStroke+1) + [bmiMedianPrestroke+x for x in range(10,50,10)]
waistMedianPrestroke = 97.5 - 1.618*10
waistList = [waistMedianPrestroke] * (indexStroke+1) + [waistMedianPrestroke+x for x in range(10,50,10)]
antiHypertensiveCountList = [0]*7
statinList = [1]*7
otherLipidLoweringMedicationCountList = [0]*7
creatinineList = [1]*7
anyPhysicalActivityList=[1]*7
medianGcpPrestroke = 6.104780222+52.7
gcpList = [medianGcpPrestroke]*7

In [18]:
class TestCaseOne(Person):
    
    def __init__(self):
        #super(gfrForTestCaseOne, self).__init__()
        super().__init__(
            age= ageList[0],                                          #agemed10
            gender=NHANESGender.FEMALE,                               #female0
            raceEthnicity=NHANESRaceEthnicity.NON_HISPANIC_WHITE,     #black
            sbp= sbpList[0],                                          #bs_sbpstkcog
            dbp= dbpList[0],                                          #same as TestGCPModel      
            a1c=a1cList[0]                 ,                          #bs_glucosefmed10
            hdl= hdlList[0],                                          #same as TestGCPModel
            totChol=totCholList[0],                                   #?? hdl+ldl+0.2*trig
            ldl= ldlList[0],                                          #bs_cholldlmed10
            trig=trigList[0],                                         #same as TestGCPModel
            bmi= bmiList[0],                                          #bs_bmimed
            waist= waistList[0],                                      #bs_waistcmmed10
            anyPhysicalActivity=anyPhysicalActivityList[0],           #physact
            education=Education.SOMEHIGHSCHOOL,                       #educ2,educ3,educ4
            smokingStatus=SmokingStatus.NEVER,                        #currsmoker
            alcohol=AlcoholCategory.NONE,                             #alcperwk
            antiHypertensiveCount=antiHypertensiveCountList[0],       #htntx
            statin=statinList[0],                                     #choltx
            otherLipidLoweringMedicationCount=otherLipidLoweringMedicationCountList[0],#same as TestGCPModel
            creatinine=creatinineList[0],                             #same as TestGCPModel
            initializeAfib=TestGCPStrokeModel.initializeAfib,         #hxafib
           )
        
        self._age = ageList[0:3]
        self.add_outcome_event(Outcome(OutcomeType.STROKE, False))
        self._age = ageList
        self._sbp = sbpList
        self._dbp = dbpList
        self._a1c = a1cList
        self._hdl = hdlList
        self._totChol = totCholList
        self._ldl = ldlList
        self._trig = trigList
        self._bmi = bmiList
        self._waist = waistList
        self._anyPhysicalActivity = anyPhysicalActivityList
        self._antiHypertensiveCount = antiHypertensiveCountList
        self._statin = statinList
        self._otherLipidLoweringMedicationCount = otherLipidLoweringMedicationCountList
        self._creatinine = creatinineList
        self._gcp = gcpList
    
    @property
    def _gfr(self):
        return 80.61366 #override the gfr calculation and use the measurement for this person

test_case_one = TestCaseOne()
#test_case_one._gfr, test_case_one._age

In [19]:
ageAtLastStroke=test_case_one.get_age_at_last_outcome(OutcomeType.STROKE)
#the get_wave function gives me the wave that follows the updates to the person object
#in other words, this function returns the wave that the person is currently in,
#after the last updates, but I want the wave that gave me the last updates
waveAtLastStroke=test_case_one.get_wave_for_age(ageAtLastStroke)-1
#ageAtLastStroke, waveAtLastStroke

In [20]:
test_case_one._outcomes

{<OutcomeType.MI: 'mi'>: [],
 <OutcomeType.STROKE: 'stroke'>: [(78.49117043,
   Outcome type: OutcomeType.STROKE, fatal: False)],
 <OutcomeType.DEMENTIA: 'dementia'>: []}

In [21]:
#test_case_one._age[-1], ageAtLastStroke

In [22]:
#this should give me what I got above
(GCPStrokeModel()
 .calc_linear_predictor_for_patient_characteristics(
        ageAtLastStroke = ageAtLastStroke,
        yearsSinceStroke = test_case_one._age[-1]-ageAtLastStroke,
        gender = test_case_one._gender,
        raceEthnicity = test_case_one._raceEthnicity,
        education = test_case_one._education,
        smokingStatus = test_case_one._smokingStatus,
        #diabetes = test_case_one.has_diabetestx(),
        physicalActivity = test_case_one._anyPhysicalActivity[-1],
        alcoholPerWeek = test_case_one._alcoholPerWeek[-1],
        medianBmiPrestroke = np.median(np.array(test_case_one._bmi[:waveAtLastStroke+1])),
        meanSBP = np.array(test_case_one._sbp[waveAtLastStroke+1:]).mean(),
        meanSBPPrestroke = np.array(test_case_one._sbp[:waveAtLastStroke+1]).mean(),
        meanLdlPrestroke = np.array(test_case_one._ldl[:waveAtLastStroke+1]).mean(),
        meanLdl = np.array(test_case_one._ldl[waveAtLastStroke+1:]).mean(),
        #cannot control this property for now
        gfr = test_case_one._gfr,
        medianWaistPrestroke = np.median(np.array(test_case_one._waist[:waveAtLastStroke+1])),
        meanFastingGlucose = Person.convert_a1c_to_fasting_glucose(np.array(test_case_one._a1c[waveAtLastStroke+1:]).mean()),
        meanFastingGlucosePrestroke = Person.convert_a1c_to_fasting_glucose(np.array(test_case_one._a1c[:waveAtLastStroke+1]).mean()),
        anyAntiHypertensive = test_case_one._current_bp_treatment,
        anyLipidLowering = test_case_one._statin[-1] | (test_case_one._otherLipidLoweringMedicationCount[-1]>0),
        afib = test_case_one._afib[-1],
        mi= test_case_one._mi,
        medianGCPPrestroke = np.median(np.array(test_case_one._gcp[:waveAtLastStroke+1])))    
)

51.87609553705786

### REPRODUCE RESULTS USING get_risk_for_person (non-vectorized) AND A PERSON

In [32]:
#seedSequence = np.random.SeedSequence()
#rngStream = np.random.default_rng(seed=seedSequence)

#should be same as above
(GCPStrokeModel().get_risk_for_person(test_case_one, 
                                      rng=None, 
                                      years=1,
                                      vectorized=False,
                                      test=True))

51.87609553705786

### REPRODUCE RESULTS USING get_risk_for_person (vectorized) AND A PERSON-ROW

In [34]:
pop = Population(pd.Series(test_case_one))
df = pop.get_people_current_state_and_summary_as_dataframe()

#should be same as above
GCPStrokeModel().get_risk_for_person(person=df.iloc[0], rng=None, 
                                      years=1,
                                      vectorized=True,
                                      test=True)

51.87609553705786

### TROUBLESHOOTING

In [ ]:
#use this for troubleshooting
person=alive.iloc[0]

ageAtLastStroke=person.ageAtLastStroke
(
#ageAtLastStroke,
#person.age-ageAtLastStroke, test_case_one._age[-1]-ageAtLastStroke,
#person.gender, test_case_one._gender,
#person.raceEthnicity, test_case_one._raceEthnicity,
#person.education, test_case_one._education,
#person.smokingStatus, test_case_one._smokingStatus,
#person.current_diabetes, test_case_one.has_diabetes(),
#person.anyPhysicalActivity, test_case_one._anyPhysicalActivity[-1],
#person.alcoholPerWeek, test_case_one._alcoholPerWeek[-1],
#person.medianBmiPriorToLastStroke, np.median(np.array(test_case_one._bmi[:waveAtLastStroke+1])),
person.meanSbpSinceLastStroke, np.array(test_case_one._sbp[waveAtLastStroke+1:]).mean(), 
#person.meanSbpPriorToLastStroke, np.array(test_case_one._sbp[:waveAtLastStroke+1]).mean(),
#person.meanLdlPriorToLastStroke, np.array(test_case_one._ldl[:waveAtLastStroke+1]).mean(),
person.meanLdlSinceLastStroke, np.array(test_case_one._ldl[waveAtLastStroke+1:]).mean(),
    np.array(test_case_one._ldl[:]).mean(),
#person.gfr, test_case_one._gfr,
#person.medianWaistPriorToLastStroke, np.median(np.array(test_case_one._waist[:waveAtLastStroke+1])),
Person.convert_a1c_to_fasting_glucose(person.meanA1cSinceLastStroke), Person.convert_a1c_to_fasting_glucose(np.array(test_case_one._a1c[waveAtLastStroke:]).mean()),
Person.convert_a1c_to_fasting_glucose(np.array(test_case_one._a1c[:]).mean()),
#Person.convert_a1c_to_fasting_glucose(person.meanA1cPriorToLastStroke), Person.convert_a1c_to_fasting_glucose(np.array(test_case_one._a1c[:waveAtLastStroke+1]).mean()),
#((person.antiHypertensiveCount + person.totalBPMedsAdded)> 0), test_case_one._current_bp_treatment,
#(person.statin | (person.otherLipidLoweringMedicationCount>0.)), test_case_one._statin[-1] | (test_case_one._otherLipidLoweringMedicationCount[-1]>0),
#person.afib, test_case_one._afib[-1],
#person.mi, test_case_one._mi,
#person.medianGcpPriorToLastStroke, np.median(np.array(test_case_one._gcp[:waveAtLastStroke+1])))
)

In [ ]:
Person.convert_a1c_to_fasting_glucose(test_case_two._a1c[3])

In [ ]:
test_case_two._age

In [ ]:
ageAtStroke = 74.6 + -1.21225*10
indexStroke = 2
ageList = [ageAtStroke + 3.3976728 + x for x in range(-5,1)]
ageList.insert(indexStroke, ageAtStroke)
ageList

In [ ]:
Person.convert_a1c_to_fasting_glucose(test_case_three._a1c[0])

In [ ]:
ageAtLastStroke=test_case_three.get_age_at_last_outcome(OutcomeType.STROKE)
#the get_wave function gives me the wave that follows the updates to the person object
#in other words, this function returns the wave that the person is currently in,
#after the last updates, but I want the wave that gave me the last updates
waveAtLastStroke=test_case_three.get_wave_for_age(ageAtLastStroke)-1
#ageAtLastStroke, waveAtLastStroke

In [ ]:
ageAtStroke = 74.6 + 0.85400*10
indexStroke = 2
ageList = [ageAtStroke + 8.265571 + x for x in range(-10,1)]
ageList.insert(indexStroke, ageAtStroke) 
ageList

In [ ]:
ageAtStroke = 74.6 + 0.85400*10
        indexStroke = 2
        ageList = [ageAtStroke + 8.265571 + x for x in range(-5,1)]
        ageList.insert(indexStroke, ageAtStroke)